In [9]:

import pandas as pd
import numpy as np
from check_eventids import check_eventids, check_extension_eventids
import requests
import io
NaN = np.nan


In [6]:
link = 'https://catalogue.preprod.ogsl.ca/data/explos-nature/ca-cioos_628eafae-e4db-4524-9353-cace173ec941/explos-nature_eventcore_2018.csv'
urldata = requests.get(link)
type_encodage = urldata.apparent_encoding
urlData = urldata.content
payload = urlData.decode(type_encodage)

event = pd.read_csv(io.StringIO( payload))  
check_eventids(event)


,field,level,row,message


In [7]:
link = 'https://catalogue.preprod.ogsl.ca/data/explos-nature/ca-cioos_628eafae-e4db-4524-9353-cace173ec941/explos-nature_occurrence_2019.csv'
urldata = requests.get(link)
type_encodage = urldata.apparent_encoding
urlData = urldata.content
payload = urlData.decode(type_encodage)

extension = pd.read_csv(io.StringIO( payload))  
extension.head(2)

,eventID,occurrenceID,eventDate,decimalLatitude,decimalLongitude,taxonRank,scientificName,scientificNameID,occurrenceStatus,occurrenceRemark,basisOfRecord,organismRemark
0,EXPLOS-NAT_EVENT_2019_section_8_1_30_25,EXPLOS-NAT_OCCURRENCE_2019_section_8_1_30_25_1,2019-09-10,48.50225,-69.2261,Species,Metridium senile,urn:lsid:marinespecies.org:taxname:100982,absence,denombrement,HumanObservation,NaN
1,EXPLOS-NAT_EVENT_2019_section_8_1_30_25,EXPLOS-NAT_OCCURRENCE_2019_section_8_1_30_25_2,2019-09-10,48.50225,-69.2261,Species,Urticina felina,urn:lsid:marinespecies.org:taxname:100834,absence,denombrement,HumanObservation,NaN


In [19]:
event = pd.DataFrame(data=event)
event = event.replace('', NaN)
event.rename(columns=str.lower, inplace=True)
column_names = list(event.columns)

extension = pd.DataFrame(data=extension)
extension = extension.replace('', NaN)
extension.rename(columns=str.lower, inplace=True)
field = 'eventID'

if 'eventid' in column_names:
    field = field.lower()
    # event.rename(columns=str.lower, inplace=True)
    # extension.rename(columns=str.lower, inplace=True)

    extension_eventids = extension[field]
    event_eventids = event['eventid']

    field_analysis = pd.DataFrame(columns=['field', 'level', 'row', 'message'])

    extension_eventids = pd.DataFrame(data = extension_eventids)
    extension_eventids.loc[:, 'message'] = extension_eventids[field].isin(event_eventids)

    extension_eventids = extension_eventids[~extension_eventids['message']]

    if len(extension_eventids) != 0:
        field_analysis['field'] = extension_eventids[field]
        field_analysis['level'] = 'error'
        field_analysis['row'] = extension_eventids.index
        field_analysis['message'] = field_analysis.agg('Field {0[field]} has no corresponding eventID in the core'.format, axis=1)
        field_analysis['field'] = field

field_analysis

,field,level,row,message
0,eventid,error,0,Field EXPLOS-NAT_EVENT_2019_section_8_1_30_25 ...
1,eventid,error,1,Field EXPLOS-NAT_EVENT_2019_section_8_1_30_25 ...
2,eventid,error,2,Field EXPLOS-NAT_EVENT_2019_section_8_1_30_25 ...
3,eventid,error,3,Field EXPLOS-NAT_EVENT_2019_section_8_1_30_25 ...
4,eventid,error,4,Field EXPLOS-NAT_EVENT_2019_section_8_1_30_25 ...
...,...,...,...,...
12392,eventid,error,12392,Field EXPLOS-NAT_EVENT_2019_photoquadrat_8_7_8...
12393,eventid,error,12393,Field EXPLOS-NAT_EVENT_2019_photoquadrat_8_7_9...
12394,eventid,error,12394,Field EXPLOS-NAT_EVENT_2019_photoquadrat_8_8_1...
12395,eventid,error,12395,Field EXPLOS-NAT_EVENT_2019_photoquadrat_8_8_7...


In [20]:
extension_eventids[field]

0         EXPLOS-NAT_EVENT_2019_section_8_1_30_25
1         EXPLOS-NAT_EVENT_2019_section_8_1_30_25
2         EXPLOS-NAT_EVENT_2019_section_8_1_30_25
3         EXPLOS-NAT_EVENT_2019_section_8_1_30_25
4         EXPLOS-NAT_EVENT_2019_section_8_1_30_25
                           ...                   
12392    EXPLOS-NAT_EVENT_2019_photoquadrat_8_7_8
12393    EXPLOS-NAT_EVENT_2019_photoquadrat_8_7_9
12394    EXPLOS-NAT_EVENT_2019_photoquadrat_8_8_1
12395    EXPLOS-NAT_EVENT_2019_photoquadrat_8_8_7
12396    EXPLOS-NAT_EVENT_2019_photoquadrat_8_8_8
Name: eventid, Length: 12397, dtype: object